In [1]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import copy
import glob
import numpy as np

In [2]:
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b55-1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        print('creating file...')
        variable_to_store = "11e1d6e4-74ba-49cc-9b55-1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')
def adjust_identifier(root_copy,filepath_base_identifier):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
        return 'cac'.join(t_split)

In [3]:

sheet_name = '7_Bulk Submission'

output_folder = '/Users/cheongray/iras_api_clean_1/' + sheet_name
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

output_file = os.path.join(output_folder, 'df_full.csv')
output_file_original = os.path.join(output_folder, 'df_full_back_up.csv')
if os.path.exists(output_file):
    os.remove(output_file)
if os.path.exists(output_file_original):
    os.remove(output_file_original)


In [4]:
file_path = r"/Users/cheongray/iras_api_pytest_1/AP_SP_files/AP_SP_FINAL_5.xlsx"#download to here from emails
filepath_base_identifier = '/Users/cheongray/iras_api_make_xml/identifier_store/stored_base_identifier_conditional_validation.txt'
path_to_sample_xml_files_folder = r"/Users/cheongray/iras_api_clean_1/sample_xml_files"

df_s = pd.read_excel(file_path, sheet_name=sheet_name , keep_default_na=False, na_values=['NA'])
df_s_temp = df_s.copy()
df_s_temp = df_s_temp.fillna('NA')
df_s_temp['Test Case ID'] = df_s_temp.index.astype(float)#REMINDER,now test_case_id is 1_1
# df_s_peppol = df_s_temp[(df_s_temp['Standard'] == 'PEPPOL') & (df_s_temp['Document Type'] == 'Invoice')].copy()
# df_s_peppol['Test Case ID'] = df_s_peppol['Test Case ID'] + 1
# df_s_peppol.reset_index(drop=True)
df_s_temp['Test Case ID'] = df_s_temp['Test Case ID'] + 1
df_s_temp.to_csv(output_file, index=False)
df_s_temp.to_csv(output_file_original, index=False)

In [5]:

df_s_temp_subset = df_s_temp.copy()

unique_descriptions = set()

target_elements_syn = []

for index,row in df_s_temp_subset.iterrows():
    filled_columns = row.dropna().index
    test_case_id = f' {index}'
    descript = row['Test Case Description']
#     print(f'Test case {index+1}')
#     print(f'Test case description : {descript}')
    unique_descriptions.add(descript)#just to build function_mapping
    temp={}
    temp['Test Case ID'] = int(row['Test Case ID'])
    id = int(row['Test Case ID'])
    temp['descript'] = descript
    temp['Expected Result'] = row['Expected Result Response (for 400 response)']
    temp['Status'] = row['Expected Result']
    temp['Fields'] = []
    temp['Sample'] = row['Sample']
    for i, col in enumerate(filled_columns, start=1):
        field_name = f'Field {i}'
        if field_name in df_s_temp_subset.columns:
            # Process the data as needed
#             print(f'Column: {field_name}, Value: {row[field_name]}')
   
            
            temp_2 = [] #each field will be one array,can do dict but mor steps when want access
            if not pd.isna(row[field_name]):
                field_value = field_name + ' Value'
                field_name_insert = fix_space_typo(row[field_name])
                temp_2.append(field_name_insert)
                temp_2.append(row[field_value])
                if field_name_insert == '':
                    continue
                else:
                    temp['Fields'].append(temp_2)
        
    target_elements_syn.append(temp)

            


In [6]:
folder_name = output_folder + '/xml_files_for'+'_'+sheet_name
folder_name

'/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission'

In [7]:
# path_to_sample_xml_files = '/Users/cheongray/iras_api_pytest_1/sample_xml_files'
#gets the needed files for 2_Conditional_Input_Validation
file_paths_xml_files = {}

for i,test_case in enumerate(target_elements_syn):
    
    sample_to_use= str(test_case['Sample'])
    print(sample_to_use)
    
    file_name = 'sample' + '_' + sample_to_use+ '.xml'
    
    path_to_sample_xml_files = path_to_sample_xml_files_folder + '/' + file_name
    
    file_paths_xml_files[sample_to_use] = path_to_sample_xml_files


21
22
23
24
25
26
27
28
29


In [8]:
file_paths_xml_files

{'21': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_21.xml',
 '22': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_22.xml',
 '23': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_23.xml',
 '24': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_24.xml',
 '25': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_25.xml',
 '26': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_26.xml',
 '27': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_27.xml',
 '28': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_28.xml',
 '29': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_29.xml'}

In [9]:
folder_name

'/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission'

In [10]:
target_elements_syn

[{'Test Case ID': 1,
  'descript': 'Multiple invoices in one single submission (<=10)',
  'Expected Result': '',
  'Status': 200,
  'Fields': [],
  'Sample': 21},
 {'Test Case ID': 2,
  'descript': 'Multiple credit notes in one single submission (<=10)',
  'Expected Result': '',
  'Status': 200,
  'Fields': [],
  'Sample': 22},
 {'Test Case ID': 3,
  'descript': 'Multiple invoices and credit notes in one single submission',
  'Expected Result': '',
  'Status': 400,
  'Fields': [],
  'Sample': 23},
 {'Test Case ID': 4,
  'descript': 'Multiple invoices in one single submission (>10)',
  'Expected Result': '',
  'Status': 400,
  'Fields': [],
  'Sample': 24},
 {'Test Case ID': 5,
  'descript': 'Multiple credit notes in one single submission (>10)',
  'Expected Result': '',
  'Status': 400,
  'Fields': [],
  'Sample': 25},
 {'Test Case ID': 6,
  'descript': 'PEPPOL invoice(s) within the body',
  'Expected Result': '',
  'Status': 200,
  'Fields': [],
  'Sample': 26},
 {'Test Case ID': 7,
 

In [11]:
file_paths_xml_files

{'21': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_21.xml',
 '22': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_22.xml',
 '23': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_23.xml',
 '24': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_24.xml',
 '25': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_25.xml',
 '26': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_26.xml',
 '27': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_27.xml',
 '28': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_28.xml',
 '29': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_29.xml'}

In [57]:
destination_folder = folder_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)
for index,test_case in enumerate(target_elements_syn):
    test_case_id = test_case['Test Case ID']
    sample_key = str(test_case['Sample'])
    sample_path = file_paths_xml_files[sample_key]
    destination_path = os.path.join(destination_folder, f"{test_case_id}_{os.path.basename(sample_path)}")
    print(destination_path)
    shutil.copy(sample_path, destination_path)

/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/1_sample_21.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/2_sample_22.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/3_sample_23.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/4_sample_24.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/5_sample_25.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/6_sample_26.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/7_sample_27.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/8_sample_28.xml
/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/9_sample_29.xml


In [58]:
destination_folder

'/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission'

In [59]:
xml_files = [f for f in os.listdir(destination_folder) if f.endswith('.xml')]
for file in xml_files:
    t_id = file.split('_')[0]
    print(t_id)

1
6
2
5
9
8
3
4
7


In [60]:
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
# if os.path.exists(folder_name):
#     shutil.rmtree(folder_name)
#     os.makedirs(folder_name)

# destination_folder = folder_name

# for key, file_path in file_paths_xml_files.items():
# #     print(file_path)
#     if os.path.exists(file_path):
#         destination_path = os.path.join(destination_folder, os.path.basename(file_path))
#         print(destination_path)
# #         print(destination_path)
# #         shutil.copy(file_path, destination_path)
# #         print(f"Copied {file_path} to {destination_path}")
# #     else:
# #         print(f"File {file_path} does not exist")

In [61]:
destination_folder

'/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission'

In [62]:
load_base_value_iden(filepath_base_identifier)

'11checking_test-74ba-49cc-9b55-13052'

In [63]:
xml_files = glob.glob(os.path.join(destination_folder, '*.xml'))

In [64]:
def parse_xml(filepath_base_identifier):
    # Parse the XML file
    tree = etree.parse(filepath_base_identifier)
    # Get the root element
    root = tree.getroot()
    
    # Process the XML tree without printing
    return root    

In [65]:
for f in xml_files:
    root = parse_xml(f)
    base_identifier = load_base_value_iden(filepath_base_identifier)
    tree = etree.ElementTree(root)
    for index,e in enumerate(root.iter()):   
        if index == 10:
            print(f'original identifier{e.text}')
            e.text = base_identifier
            print(f'new identifier{e.text}')
            increment_value_iden(base_identifier,filepath_base_identifier) 
            tree.write(f, encoding='utf-8', xml_declaration=True)
            

original identifier11e1d6e4-74ba-49cc-9b56-b7a3a7204888
new identifier11checking_test-74ba-49cc-9b55-13052
original identifier66e1d6e4-74ba-49cc-9b56-b7a3a10497279
new identifier11checking_test-74ba-49cc-9b55-13053
original identifier22e1d6e4-74ba-49cc-9b56-b7a3a999999
new identifier11checking_test-74ba-49cc-9b55-13054
original identifier55e1d6e4-74ba-49cc-9b56-b7a3a02850673
new identifier11checking_test-74ba-49cc-9b55-13055
original identifier99e1d6e4-74ba-49cc-9b56-b7a3af836f9
new identifier11checking_test-74ba-49cc-9b55-13056
original identifier88e1d6e4-74ba-49cc-9b56-b7a3a37596792
new identifier11checking_test-74ba-49cc-9b55-13057
original identifier33e1d6e4-74ba-49cc-9b56-b7a3a58204856
new identifier11checking_test-74ba-49cc-9b55-13058
original identifier44e2d6e4-74ba-49cc-9b57-b7a3a64067220
new identifier11checking_test-74ba-49cc-9b55-13059
original identifier77e1d6e4-74ba-49cc-9b56-b7a3a8305729
new identifier11checking_test-74ba-49cc-9b55-13060


In [66]:
xml_files

['/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/1_sample_21.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/6_sample_26.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/2_sample_22.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/5_sample_25.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/9_sample_29.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/8_sample_28.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/3_sample_23.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/4_sample_24.xml',
 '/Users/cheongray/iras_api_clean_1/7_Bulk Submission/xml_files_for_7_Bulk Submission/7_sample_27.xml']